# Atividade 02: Atributos Categóricos e Valores Faltantes
### Aluno: Alisson da Silva Vieira

# Bibliotecas utilizadas
- Numpy: É uma biblioteca fundamental para computação científica em Python, que fornece um objeto de matriz multidimensional, vários objetos derivados (como matrizes e matrizes mascaradas) e uma variedade de rotinas para operações rápidas em matrizes.
- Pandas: É uma biblioteca que fornece estruturas de dados rápidas, flexíveis e expressivas projetadas para tornar o trabalho com dados "relacionais" ou "rotulados" fácil e intuitivo. Tem como objetivo ser o bloco de construção fundamental de alto nível para fazer análises de dados.
- Scikit-learn: É uma biblioteca em Python para aprendizado de máquina desenvolvido com base no SciPy que fornece um conjunto de ferramentas de machine learning.
- Scipy: É um módulo que fornece uma coleção de algoritmos matemáticos e funções de conveniência baseadas na extensão NumPy do Python. Ele adiciona um poder significativo à sessão Python interativa, fornecendo ao usuário comandos e classes de alto nível para manipulação e visualização de dados.
- tqdm: É uma biblioteca Python para acompanhar o progresso de um processo iterativo.

In [67]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from tqdm.notebook import tqdm
from scipy.stats import ttest_ind_from_stats
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

%matplotlib inline

Inicialmente abrimos o arquivo .csv, e conseguimos analisar os primeiros 5 dados do arquivo.

In [68]:
df = pd.read_csv('data/agaricus_lepiota_small_c.csv')
df.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,e,x,s,y,t,a,f,w,b,g,...,s,w,w,p,w,o,p,n,v,d
1,e,f,s,y,f,n,f,c,b,p,...,s,w,w,p,w,o,f,n,y,g
2,e,k,s,w,f,c,f,w,b,g,...,s,w,n,p,w,t,e,w,n,g
3,e,f,f,n,t,n,f,c,b,w,...,s,g,w,p,w,o,p,k,v,d
4,p,x,s,w,t,p,f,c,n,w,...,s,w,w,p,w,o,p,n,s,u


Conseguimos analisar também as colunas. </br>
De cara já verificamos que apenas a coluna 'stalk-root' possui valores faltantes.

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   class                     1000 non-null   object
 1   cap-shape                 1000 non-null   object
 2   cap-surface               1000 non-null   object
 3   cap-color                 1000 non-null   object
 4   bruises                   1000 non-null   object
 5   odor                      1000 non-null   object
 6   gill-attachment           1000 non-null   object
 7   gill-spacing              1000 non-null   object
 8   gill-size                 1000 non-null   object
 9   gill-color                1000 non-null   object
 10  stalk-shape               1000 non-null   object
 11  stalk-root                690 non-null    object
 12  stalk-surface-above-ring  1000 non-null   object
 13  stalk-surface-below-ring  1000 non-null   object
 14  stalk-color-above-ring   

E para uma melhor análise, conseguimos fazer uma breve análise estatística, agrupando nossos dados pela coluna 'class'. 

In [70]:
df.groupby('class').describe()

cap-shape                 cap-surface                 cap-color         \
          count unique top freq       count unique top freq     count unique   
class                                                                          
e           518      5   x  239         518      3   y  186       518     10   
p           482      4   x  222         482      3   y  225       482      8   

       ... spore-print-color      population                 habitat         \
       ...               top freq      count unique top freq   count unique   
class  ...                                                                    
e      ...                 n  208        518      6   v  153     518      7   
p      ...                 w  209        482      6   v  301     482      7   

                
      top freq  
class           
e       d  221  
p       d  171  

[2 rows x 88 columns]

Como os dados da coluna 'stalk-root' possuem valores faltantes, irei tirar essa coluna do dataset. <br>
Minha hipotese é de que como o dataset possui 21 atributos sem contar o 'stalk-root', retirar ele dos nossos dados pode não prejudicar a análise.

In [71]:
df = df.drop(['stalk-root'], axis=1)

Conseguimos perceber que todos os nossos dados possuem valores ordinais, então temos que realizar uma transformação para que os dados sejam categóricos.

In [72]:
transformers = [
    ('v_class', OrdinalEncoder(categories=[['e', 'p']]), ['class']),
    ('v_cap-shape', OrdinalEncoder(), ['cap-shape']),
    ('v_cap-surface', OrdinalEncoder(), ['cap-surface']),
    ('v_cap-color', OrdinalEncoder(), ['cap-color']),
    ('v_bruises', OrdinalEncoder(categories=[['t', 'f']]), ['bruises']),
    ('v_odor', OrdinalEncoder(), ['odor']),
    ('v_gill-attachment', OrdinalEncoder(), ['gill-attachment']),
    ('v_gill-spacing', OrdinalEncoder(categories=[['c', 'w', 'd']]), ['gill-spacing']),
    ('v_gill-size', OrdinalEncoder(categories=[['b', 'n']]), ['gill-size']),
    ('v_gill-color', OrdinalEncoder(), ['gill-color']),
    ('v_stalk-shape', OrdinalEncoder(categories=[['e', 't']]), ['stalk-shape']),
    # ('v_stalk-root', OrdinalEncoder(), ['stalk-root']),
    ('v_stalk-surface-above-ring', OrdinalEncoder(), ['stalk-surface-above-ring']),
    ('v_stalk-surface-below-ring', OrdinalEncoder(), ['stalk-surface-below-ring']),
    ('v_stalk-color-above-ring', OrdinalEncoder(), ['stalk-color-above-ring']),
    ('v_stalk-color-below-ring', OrdinalEncoder(), ['stalk-color-below-ring']),
    ('v_veil-type', OrdinalEncoder(categories=[['p', 'u']]), ['veil-type']),
    ('v_veil-color', OrdinalEncoder(), ['veil-color']),
    ('v_ring-number', OrdinalEncoder(categories=[['n', 'o', 't']]), ['ring-number']),
    ('v_ring-type', OrdinalEncoder(), ['ring-type']),
    ('v_spore-print-color', OrdinalEncoder(), ['spore-print-color']),
    ('v_population', OrdinalEncoder(), ['population']),
    ('v_habitat', OrdinalEncoder(), ['habitat'])
]

ct = ColumnTransformer(transformers=transformers)

X_oe = ct.fit_transform(df)
df_oe = pd.DataFrame(X_oe, columns=df.columns)

Esse é o resultado depois da transformação dos dados.

In [73]:
df_oe.head()

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,0.0,4.0,1.0,9.0,0.0,0.0,1.0,1.0,0.0,2.0,...,2.0,7.0,7.0,0.0,2.0,1.0,4.0,3.0,4.0,0.0
1,0.0,1.0,1.0,9.0,1.0,5.0,1.0,0.0,0.0,7.0,...,2.0,7.0,7.0,0.0,2.0,1.0,1.0,3.0,5.0,1.0
2,0.0,2.0,1.0,8.0,1.0,1.0,1.0,1.0,0.0,2.0,...,2.0,7.0,4.0,0.0,2.0,2.0,0.0,7.0,2.0,1.0
3,0.0,1.0,0.0,4.0,0.0,5.0,1.0,0.0,0.0,10.0,...,2.0,3.0,7.0,0.0,2.0,1.0,4.0,2.0,4.0,0.0
4,1.0,4.0,1.0,8.0,0.0,6.0,1.0,0.0,1.0,10.0,...,2.0,7.0,7.0,0.0,2.0,1.0,4.0,3.0,3.0,5.0


Agora, conseguimos observar melhor o tipo dos valores das colunas.

In [74]:
# pegar as labels
y_oe = df_oe['class'].values

# drop do atributo classe
df_oe.drop(['class'], axis=1, inplace=True)

X_oe = df_oe.values
df_oe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   cap-shape                 1000 non-null   float64
 1   cap-surface               1000 non-null   float64
 2   cap-color                 1000 non-null   float64
 3   bruises                   1000 non-null   float64
 4   odor                      1000 non-null   float64
 5   gill-attachment           1000 non-null   float64
 6   gill-spacing              1000 non-null   float64
 7   gill-size                 1000 non-null   float64
 8   gill-color                1000 non-null   float64
 9   stalk-shape               1000 non-null   float64
 10  stalk-surface-above-ring  1000 non-null   float64
 11  stalk-surface-below-ring  1000 non-null   float64
 12  stalk-color-above-ring    1000 non-null   float64
 13  stalk-color-below-ring    1000 non-null   float64
 14  veil-type

Abaixo se encontra as funções responsáveis pela classificação dos dados, usando o SVM e o KNN

In [75]:
'''
    ### Funções referente ao classificador KNN ####
'''

def validacaoCruzadaKnn(X, y, kVias = 10):

    print('Iniciando a validação cruzada com o classificado KNN...')

    # acuracias
    acuracias = []

    # usar o protocolo de validação cruzada estratificada
    skf = StratifiedKFold(n_splits=kVias, shuffle=True, random_state=1)

    pgb = tqdm(total=kVias, desc="Fold's avaliados")
    idx = 1

    for idx_treino, idx_teste in skf.split(X, y):

        # extrair as instâncias de treinamento de acordo com os índices fornecidos pelo skf.split
        X_treino = X[idx_treino]
        y_treino = y[idx_treino]
        
        # extrair as instâncias de teste de acordo com os índices fornecidos pelo skf.split
        X_teste = X[idx_teste]
        y_teste = y[idx_teste]

        # separar as instâncias de treinamento entre treinamento e validação para a otimização do hiperparâmetro k
        X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.2, stratify=y_treino, shuffle=True, random_state=1)

        params = {'n_neighbors' : range(1,30,2)}

        ss = StandardScaler()
        ss.fit(X_treino)
        X_treino = ss.transform(X_treino)
        X_teste = ss.transform(X_teste)
        X_val = ss.transform(X_val)

        # knn = GridSearchCV(KNeighborsClassifier(), params, cv=StratifiedKFold(n_splits=5), scoring='accuracy', n_jobs=-1)
        knn = GridSearchCV(KNeighborsClassifier(), params, cv=StratifiedKFold(n_splits=5), scoring='accuracy')
        knn.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])
        
        pred = knn.predict(X_teste)

        print('Fold ', idx, '\nf1-socre: ', round(f1_score(y_teste, pred), 2), '\nAccuracy score: ', accuracy_score(y_teste, pred), end='\n\n')

        # calcular a acurácia no conjunto de testes desta iteração e salvar na lista.
        acuracias.append(f1_score(y_teste, pred))

        pgb.update(1)
        idx+=1
        
    pgb.close()
    
    return acuracias

'''
    ### Funções referente ao classificador SVM ####
'''

def validacaoCruzadaSvm(X, y, cv_splits, Cs=[1], gammas=['scale']):

    print('Iniciando a validação cruzada com o classificador SVM...')

    parameters = [{
        'gamma': gammas,
        'C': Cs,
        'kernel': ['rbf']
    }]

    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=1)

    acuracias = []
    
    pgb = tqdm(total=cv_splits, desc="Fold's avaliados")
    idx = 1

    for treino_idx, teste_idx in skf.split(X, y):

        X_treino = X[treino_idx]
        y_treino = y[treino_idx]

        X_teste = X[teste_idx]
        y_teste = y[teste_idx]

        X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, stratify=y_treino, test_size=0.2, random_state=1)

        ss = StandardScaler()
        ss.fit(X_treino)
        X_treino = ss.transform(X_treino)
        X_teste = ss.transform(X_teste)
        X_val = ss.transform(X_val)

        svm = GridSearchCV(SVC(), parameters, cv=StratifiedKFold(n_splits=5))
        svm.fit(np.vstack((X_treino, X_val)), [*y_treino, *y_val])
        
        pred = svm.predict(X_teste)

        print('Fold ', idx, '\nf1-socre: ', round(f1_score(y_teste, pred), 2), '\nAccuracy score: ', accuracy_score(y_teste, pred),  end='\n\n')

        acuracias.append(f1_score(y_teste, pred))

        pgb.update(1)
        idx+=1
        
    pgb.close()
    
    return acuracias

'''
    ### Funções auxiliares ####
'''

def showResult(acc, legend):
    print('Resultado ', legend, '\n    >> Acc mínima: ', round(min(acc), 3), '%\n    >> Acc máxima: ', round(max(acc), 3), '%')
    print('    >> Média: ', round(np.mean(acc), 3), '\n    >> Desvio padrão: ', round(np.std(acc), 3), '\n')

    return np.mean(acc), np.std(acc)

def hipoteseNula(media1, std1, values1, media2, std2, values2, alpha=0.05):
    pvalor = ttest_ind_from_stats(media1, std1, len(values1), media2, std2, len(values2))[1]
    
    return pvalor <= alpha

### Os resultados obtidos

Os resultados para cada fold é mostrado abaixo, utilizando as métricas: 'f1-score' e 'accuracy score'.

In [76]:
accKnn = validacaoCruzadaKnn(X_oe, y_oe)

Iniciando a validação cruzada com o classificado KNN...


Fold's avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

Fold  1 
f1-socre:  0.92 
Accuracy score:  0.92

Fold  2 
f1-socre:  0.87 
Accuracy score:  0.88

Fold  3 
f1-socre:  0.9 
Accuracy score:  0.91

Fold  4 
f1-socre:  0.89 
Accuracy score:  0.89

Fold  5 
f1-socre:  0.95 
Accuracy score:  0.95

Fold  6 
f1-socre:  0.91 
Accuracy score:  0.91

Fold  7 
f1-socre:  0.91 
Accuracy score:  0.91

Fold  8 
f1-socre:  0.91 
Accuracy score:  0.92

Fold  9 
f1-socre:  0.91 
Accuracy score:  0.91

Fold  10 
f1-socre:  0.91 
Accuracy score:  0.91



In [77]:
accSvm = validacaoCruzadaSvm(X_oe, y_oe, 10, Cs=[1, 10, 100, 1000], gammas=['scale', 'auto', 2e-2, 2e-3, 2e-4])

Iniciando a validação cruzada com o classificador SVM...


Fold's avaliados:   0%|          | 0/10 [00:00<?, ?it/s]

Fold  1 
f1-socre:  0.9 
Accuracy score:  0.9

Fold  2 
f1-socre:  0.87 
Accuracy score:  0.88

Fold  3 
f1-socre:  0.92 
Accuracy score:  0.92

Fold  4 
f1-socre:  0.88 
Accuracy score:  0.89

Fold  5 
f1-socre:  0.91 
Accuracy score:  0.91

Fold  6 
f1-socre:  0.91 
Accuracy score:  0.91

Fold  7 
f1-socre:  0.91 
Accuracy score:  0.92

Fold  8 
f1-socre:  0.91 
Accuracy score:  0.91

Fold  9 
f1-socre:  0.89 
Accuracy score:  0.9

Fold  10 
f1-socre:  0.9 
Accuracy score:  0.9



## Análise da hípotese nula

In [78]:
mediaKnn, stdKnn = showResult(accKnn, 'knn')
mediaSvm, stdSvm = showResult(accSvm, 'svm')

hpnula = hipoteseNula(mediaKnn, stdKnn, accKnn, mediaSvm, stdSvm, accSvm)

if hpnula:
    if mediaKnn > mediaSvm:
        classif = ' knn '
    else:
        classif = ' svm '

    text = 'conseguimos rejeitar, ou seja, podemos dizer que o' + classif + 'obteve um resultado melhor.'
else:
    text = 'não podemos rejeitar, ou seja, não conseguimos afirmar que os dois resultados são estatisticamente diferentes.'

print('Hipotese nula:', text)

Resultado  knn 
    >> Acc mínima:  0.87 %
    >> Acc máxima:  0.947 %
    >> Média:  0.907 
    >> Desvio padrão:  0.019 

Resultado  svm 
    >> Acc mínima:  0.872 %
    >> Acc máxima:  0.917 %
    >> Média:  0.899 
    >> Desvio padrão:  0.013 

Hipotese nula: não podemos rejeitar, ou seja, não conseguimos afirmar que os dois resultados são estatisticamente diferentes.


## Análise final dos resultados

Após analisar os resultados, podemos ver que a acurácia de ambos os classificadores são altas, a media de todos se encontram acima de 90%. Também podemos ver o teste da hipótese nula, que para esse caso, não podemos rejeitá-la, ou seja, tanto o resultado do Knn quanto o resultado do Svm são estatisticamente iguais. Porém é de conhecimento geral que um cogumelo venenoso pode levar uma pessoa a um coma e talvez a morte, ou seja, 90% de acurácia não passa a ser um valor tão bom para esse caso, que um falso positivo pode levar a uma morte. Então levando em conta esses valores, eu não usaria algum dos classificadores para decidir se eu iria ou não comer um cogumelo classificado por algum dos classificadores.